In [1]:
%matplotlib inline

# Notebook for training predictive models
### Import packages

In [2]:
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from tensorflow.keras.models import load_model as keras_load_model
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Model
from sklearn.pipeline import Pipeline
from keras.models import Sequential

import tensorflow as tf
import pandas as pd
import numpy as np
import random
import glob
import os

from utils import load_processed_frames, split_match_ids, extract_variables, load_tf_model, evaluate_model, write_testing_error, run_model, test_model, print_column_variance, add_pred_error, add_can_be_sequentialized
from visualize_game import visualize_prediction_animation, visualize_game_animation
from settings import *

2024-05-08 12:48:39.499096: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


## Define NAIVE models

In [3]:
# NAIVE: Always predict that all players will stand still
# The calculations are based on x, y
def predict_two_seconds_naive_static(frames_df):
    frames_df['x_future_pred'] = frames_df['x']
    frames_df['y_future_pred'] = frames_df['y']

# NAIVE: Always predict that all players will continue with the same velocity
# The calculations are based on x, y, v_x, and v_y
def predict_two_seconds_naive_velocity(frames_df):
    frames_df['x_future_pred'] = frames_df['x'] + frames_df['v_x'] * seconds_into_the_future
    frames_df['y_future_pred'] = frames_df['y'] + frames_df['v_y'] * seconds_into_the_future

    # Clip values to stay on the pitch
    frames_df['x_future_pred'] = frames_df['x_future_pred'].clip(lower=0, upper=pitch_length)
    frames_df['y_future_pred'] = frames_df['y_future_pred'].clip(lower=0, upper=pitch_width)

    # Smooth the predicted coordinates
    smooth_predictions_xy(frames_df, alpha=0.95)

# NAIVE: Always predict that all players will continue with the same velocity and acceleration
# The calculations are based on x, y, v_x, v_y, a_x, and a_y
def predict_two_seconds_naive_acceleration(frames_df):
    # Calculate future positions using kinematic equationsnaive_
    frames_df['x_future_pred'] = frames_df['x'] + frames_df['v_x'] * seconds_into_the_future + 0.5 * frames_df['a_x'] * (seconds_into_the_future ** 2)
    frames_df['y_future_pred'] = frames_df['y'] + frames_df['v_y'] * seconds_into_the_future + 0.5 * frames_df['a_y'] * (seconds_into_the_future ** 2)

    # Clip values to stay on the pitch
    frames_df['x_future_pred'] = frames_df['x_future_pred'].clip(lower=0, upper=pitch_length)
    frames_df['y_future_pred'] = frames_df['y_future_pred'].clip(lower=0, upper=pitch_width)

    # Smooth the predicted coordinates
    smooth_predictions_xy(frames_df, alpha=0.95)

### Helper functions

In [4]:
# Find a frame with approximatly the same error as the average_pred_error, with an interval
def find_frame_with_average_error(frames_df, average_pred_error, error_margin):
    # For all frames
    frames = frames_df['frame'].unique()
    for frame in frames:
        current_error = frames_df[frames_df['frame'] == frame]['pred_error'].mean()
        # If the current error is within the error_margin,
        if (current_error >= average_pred_error - error_margin) and (current_error <= average_pred_error + error_margin):
            # Return the result
            return frame

    # If no frame was found
    print(f"No frame found within the error margin of {error_margin}")
    return None

# Use a naive model to make predictions on a set of games, and calculate the error
def predict_and_evaluate_naive_model(naive_model, frames_dfs, include_ball=False, ball_has_to_be_in_motion=True):
    # Concatenate the frames DataFrames into a single large DataFrame
    frames_concatenated_df = pd.concat(frames_dfs, ignore_index=True)

    # Use the custom function to make the predictions
    naive_model(frames_concatenated_df)

    # Calculate error
    error = total_error_loss(frames_concatenated_df, include_ball, ball_has_to_be_in_motion)

    return error

## Evaulate NAIVE models
### Visualize prediction errors

In [5]:
# # Visualize the predictions of the naive velocity model in an animation
# test_frames_df = train_frames_dfs[3]
# predict_two_seconds_naive_static(test_frames_df)
# total_error_loss(test_frames_df)
# visualize_prediction_animation(test_frames_df, 250, 750, "naive_static")

### Evaulate the NAIVE models with different parameters

In [6]:
# # Define the prediction functions (models) you want to test
# prediction_functions = {
#     "Naive Static": predict_two_seconds_naive_static,
#     "Naive Velocity": predict_two_seconds_naive_velocity,
#     "Naive Acceleration": predict_two_seconds_naive_acceleration
# }

# # Define the combinations of include_ball and ball_has_to_be_in_motion
# # combinations = [(True, True), (True, False), (False, True), (False, False)]
# combinations = [(False, True)]

# # Initialize an empty list to store the results
# results = []

# # Loop through each combination
# for include_ball, ball_has_to_be_in_motion in combinations:
#     # Add the combination of parameters
#     result = {"Include Ball": include_ball, "Ball in Motion": ball_has_to_be_in_motion}
#     # Loop through each prediction function (model)
#     for model_name, predict_function in prediction_functions.items():
#         # Calculate error for the current prediction function (model)
#         error = predict_and_evaluate_naive_model(predict_function, test_frames_dfs, include_ball, ball_has_to_be_in_motion)
#         result[model_name] = error
    
#     # Append the results to the list
#     results.append(result)

# # Create a DataFrame from the list of results
# results_df = pd.DataFrame(results)

# # Print the resulting DataFrame
# results_df

## Evaluate NN models

In [8]:
# Visualize model prediction
def predict_and_visualize(match_ids, model_name, start_frame, end_frame):
    # Run model
    frames_df = run_model(match_ids, model_name, downsampling_factor_testing=1)
    frames_df = add_pred_error(frames_df)

    # Visualize predictions with an animation
    visualize_prediction_animation(frames_df, start_frame, end_frame, model_name)

model_name = 'LSTM_model_v8'
test_model(model_name, downsampling_factor_testing=5)

test_ids = ['eaa1aee4-1acf-4a74-acdc-016343b231cc']
predict_and_visualize(test_ids, model_name, 800, 900)

2024-05-08 12:49:31.648982: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31133 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:8a:00.0, compute capability: 7.0
